# Training keras model in tensorflow

In [20]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model, model_from_json
from keras.utils import CustomObjectScope
from keras.layers import Dense, Dropout

In [8]:
keras.__version__, tf.__version__

('2.2.4', '1.13.1')

In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [12]:
sess = tf.Session()
K.set_session(sess)

In [14]:
with CustomObjectScope({'tf': tf}):
    model = load_model('/home/coep/pkm/distillation/py3model/keras/nn4.small2.lrn.h5')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

/home/coep/venv/py3-tflowcpu/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [18]:
## Writing json object and weights ##

model.save_weights('/home/coep/pkm/distillation/py3model/keras/nn4_lrn_weights.h5')
json_str = model.to_json()

with open("/home/coep/pkm/distillation/py3model/keras/model.json", "w") as f:
    f.write(json_str)

In [23]:
## Loading Json architecture and weights ##

rdjson = open("/home/coep/pkm/distillation/py3model/keras/model.json", 'r')
strjson = rdjson.read()
rdjson.close()
with CustomObjectScope({'tf': tf}):
    mod1 = model_from_json(strjson)
    mod1.load_weights("/home/coep/pkm/distillation/py3model/keras/nn4_lrn_weights.h5")
mod1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [41]:
model.layers[2].get_weights()

[array([[[[ 1.49056152e-01, -1.34768113e-01,  1.51423007e-01, ...,
           -1.51435016e-02, -2.09911406e-01,  1.02350123e-01],
          [-5.29299909e-03, -1.72828153e-01,  6.74600080e-02, ...,
            9.49144084e-03, -7.36269802e-02, -2.80116163e-02],
          [-9.84221995e-02, -1.66645259e-01,  1.76576570e-01, ...,
           -8.98738280e-02, -3.16160247e-02,  2.74819811e-03]],
 
         [[ 3.11939176e-02, -6.52555674e-02, -1.57041132e-01, ...,
            6.28847480e-02,  1.02878734e-02, -2.26173317e-03],
          [-2.03805685e-01, -2.17020586e-01,  1.01853756e-03, ...,
            8.54506642e-02,  1.08859288e-02,  8.94757062e-02],
          [-1.10391408e-01, -2.36338690e-01, -9.43221152e-03, ...,
            4.73738909e-02, -5.14071323e-02, -6.60058260e-02]],
 
         [[ 8.75389948e-02,  1.17656374e-02, -1.05412856e-01, ...,
            9.44804549e-02, -2.67343465e-02, -4.21006000e-03],
          [ 2.84998678e-02, -2.90884197e-01,  8.19148645e-02, ...,
            1.889

In [49]:
model.layers

In [61]:
model.get_layer("lrn_1").get_weights()

[]

In [62]:
len(model.layers)

166

In [ ]:
mdel_out = model.get_layer("dense_layer")
dif1 = tf.placeholder(tf.float32, shape=(None, 128))
loss = tf.math.square(tf.assign_sub(mdel_out, dif1))
optimizer = tf.